## 주요주주지분

In [1]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

In [2]:
cdf = pd.read_csv('data/company_name_and_code.csv', index_col = 0)
cdf

,corp_code,corp_name,stock_code
0,365387,AJ네트웍스,95570
1,125080,AK홀딩스,6840
2,219097,BGF,27410
3,1263022,BGF리테일,282330
4,858364,BNK금융지주,138930
...,...,...,...
726,111421,휴니드테크놀러지스,5870
727,362238,휴비스,79980
728,156488,휴스틸,5010
729,103176,흥국화재,540


In [3]:
my_key = pd.read_csv('data/dart_key.txt', header = None)[0][0]

In [4]:
my_key

'd6546705cd01f3257395ab1d45d6099f97d60e04'

In [28]:
base_url = 'https://opendart.fss.or.kr/api/hyslrSttus.xml'
surv_key = '?crtfc_key=' + my_key
corp_code = '&corp_code=' + '00' + '172936'
bsns_year = '&bsns_year=' + '2022'
reprt_code = '&reprt_code=11011'

In [29]:
url = base_url + surv_key + corp_code + bsns_year + reprt_code
url

'https://opendart.fss.or.kr/api/hyslrSttus.xml?crtfc_key=d6546705cd01f3257395ab1d45d6099f97d60e04&corp_code=00172936&bsns_year=2022&reprt_code=11011'

In [30]:
response = requests.get(url)
response

<Response [200]>

In [31]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><result><status>013</status><message>조회된 데이타가 없습니다.</message></result>'

In [23]:
soup = BeautifulSoup(response.text, "lxml")
soup

C:\Users\USER\AppData\Local\Temp\ipykernel_10372\2611772931.py:1: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><result><status>013</status><message>조회된 데이타가 없습니다.</message></result></body></html>

[<bsis_posesn_stock_qota_rt>30.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.01</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.02</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.01</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.01</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.01</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>7.36</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.00</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.01</bsis_posesn_stock_qota_rt>,
 <bsis_posesn_stock_qota_rt>0.01</bsis_posesn_stock_qota_rt>,
 <bsis_

In [124]:
def mse(cdf, my_key, start_year=2019, end_year=2024):
    """
    DART 기업 주요주주 지분율('계'만) 연도별 수집 함수
    - 데이터 없는 기업도 포함
    - 없는 경우 MSE, stock_value = 0
    결과:
        rdf_2019, rdf_2020, ... rdf_2024
    """
    base_url = 'https://opendart.fss.or.kr/api/hyslrSttus.xml'
    reprt_code = '11011'

    corp_codes = cdf['corp_code'].astype(str)
    corp_map = cdf[['corp_code', 'corp_name']].copy()
    corp_map['corp_code'] = corp_map['corp_code'].astype(str).str.zfill(8)

    year_dfs = {}

    for year in range(start_year, end_year + 1):
        print(f"\n=== {year}년 진행 ===")
        all_dfs = []

        for code in tqdm(corp_codes, desc=f"{year}년 기업 진행", ncols=100):
            code = code.zfill(8)

            params = {
                'crtfc_key': my_key,
                'corp_code': code,
                'bsns_year': str(year),
                'reprt_code': reprt_code
            }

            try:
                response = requests.get(base_url, params=params, timeout=10)
                response.raise_for_status()
            except requests.RequestException:
                continue

            soup = BeautifulSoup(response.text, "lxml")
            lists = soup.find_all('list')

            result = []

            for item in lists:
                nm_tag = item.find('nm')
                nm = nm_tag.text.strip() if nm_tag else None
                if nm != '계':
                    continue

                mse_tag = item.find('bsis_posesn_stock_qota_rt')
                stock_tag = item.find('stock_knd')

                result.append({
                    'corp_code': code,
                    'year': year,
                    'MSE': mse_tag.text.strip() if mse_tag else None,
                    'stock_value': stock_tag.text.strip() if stock_tag else None
                })

            if result:
                all_dfs.append(pd.DataFrame(result))

        # 🔹 DART 수집 결과
        if all_dfs:
            year_raw = pd.concat(all_dfs, ignore_index=True)
        else:
            year_raw = pd.DataFrame(columns=['corp_code', 'year', 'MSE', 'stock_value'])

        # 🔹 모든 기업 기준 DataFrame 생성
        base_df = corp_map.copy()
        base_df['year'] = year

        # 🔹 left merge → 없는 기업도 유지
        year_df = base_df.merge(
            year_raw,
            on=['corp_code', 'year'],
            how='left'
        )

        # 🔹 NaN → 0
        year_df['MSE'] = pd.to_numeric(year_df['MSE'], errors='coerce').fillna(0)
        year_df['stock_value'] = year_df['stock_value'].fillna(0)

        # 🔹 컬럼 정리
        year_df = year_df[['year', 'corp_name', 'MSE', 'stock_value']]

        year_dfs[year] = year_df

    return tuple(year_dfs[year] for year in range(start_year, end_year + 1))

In [125]:
rdf_2019, rdf_2020, rdf_2021, rdf_2022, rdf_2023, rdf_2024 = mse(
    cdf, my_key, start_year=2019, end_year=2024)


=== 2019년 진행 ===


2019년 기업 진행:   0%|                                                     | 0/731 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_10372\1691965228.py:38: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")
2019년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [00:57<00:00, 12.63it/s]



=== 2020년 진행 ===


2020년 기업 진행:   0%|                                                     | 0/731 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_10372\1691965228.py:38: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")
2020년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [00:57<00:00, 12.71it/s]



=== 2021년 진행 ===


...

2022년 기업 진행:   0%|                                                     | 0/731 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_10372\1691965228.py:38: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")
2022년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [00:57<00:00, 12.77it/s]



=== 2023년 진행 ===


2023년 기업 진행:   0%|                                                     | 0/731 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_10372\1691965228.py:38: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")
2023년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [00:59<00:00, 12.34it/s]



=== 2024년 진행 ===


2024년 기업 진행:   0%|                                                     | 0/731 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_10372\1691965228.py:38: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(response.text, "lxml")
2024년 기업 진행: 100%|███████████████████████████████████████████| 731/731 [00:59<00:00, 12.36it/s]


In [126]:
rdf_2019.isnull().sum()

year           0
corp_name      0
MSE            0
stock_value    0
dtype: int64

In [127]:
rdf_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1493 entries, 0 to 1492
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   year         1493 non-null   int64  
 1   corp_name    1493 non-null   object 
 2   MSE          1493 non-null   float64
 3   stock_value  1493 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 46.8+ KB


In [128]:
rdf_2019

,year,corp_name,MSE,stock_value
0,2019,AJ네트웍스,50.05,보통주
1,2019,AJ네트웍스,0.00,우선주
2,2019,AK홀딩스,64.89,보통주
3,2019,AK홀딩스,0.00,우선주
4,2019,BGF,69.98,보통주
...,...,...,...,...
1488,2019,휴스틸,0.00,우선주
1489,2019,흥국화재,79.19,보통주
1490,2019,흥국화재,0.00,-
1491,2019,흥아해운,22.62,보통주


In [129]:
l_rdf_max = []

for i in l_rdf:
    i['MSE'] = pd.to_numeric(i['MSE'], errors='coerce').fillna(0)

    rdf_max = (
        i.loc[i.groupby('corp_name')['MSE'].idxmax()]
         .reset_index(drop=True)
    )

    l_rdf_max.append(rdf_max)


In [130]:
rdf_2019_max = l_rdf_max[0]
rdf_2020_max = l_rdf_max[1]
rdf_2021_max = l_rdf_max[2]
rdf_2022_max = l_rdf_max[3]
rdf_2023_max = l_rdf_max[4]
rdf_2024_max = l_rdf_max[5]

In [139]:
for df, year in zip(max_lst, range(2019, 2025)):
    df.to_csv(
        f'data/Major_Shareholder/Major_Shareholder_{year}.csv',
        index=False
    )

In [134]:
#rdf.to_csv('data/Major_Shareholder_3.csv')

In [135]:
#rdf.to_csv('data/Major_Shareholder.csv')